# Level 3

Level 3 represents the highest level of user control and customization in Aviary's user interface.
At this level, users have full access to Python and OpenMDAO methods that Aviary calls.
They can use the complete set of Aviary's methods, functionalities, and classes to construct and fine-tune their aircraft models.
Level 3 enables users to have supreme control over every aspect of the model, including subsystems, connections, and advanced optimization techniques.

Level 3 is the most complex but specific methods defined at this level are used in levels 1 and 2, hopefully reducing user activation energy when learning more about Aviary.
This progressive approach helps users gradually enhance their analysis capabilities and adapt to more complex modeling requirements as they gain proficiency and experience.

We will show two approaches of level 3 development using two examples: The first one will implement all methods differently from what are available in `aviary/interface/methods_for_level2.py` but will follow the same steps. It will show that you don't have to have a `.csv` input file and you don't have to have a `phase_info` dictionary. The second is a `solved` mission approach that is embedded into all the methods of `aviary/interface/methods_for_level2.py`.

```{note}
For each of these examples we have set `max_iter = 0`, which means that the optimization will not run. This is done to reduce the computational time for the examples. If you want to run the optimization, you can set `max_iter = 100` or some similar value.
```

## A level 3 example: N3CC

In [level 2](onboarding_level2.ipynb), we have shown how to follow the standard steps to build an Aviary model, but sometimes you may find that the model you have in mind does not match that predefined structure. If you start a new model that cannot been embedded in those methods, we can write special methods ourselves. This example will show you how to do that.

In `aviary/validation_cases/benchmark_tests` folder, there is an N3CC aircraft full mission benchmark test `test_FLOPS_based_full_mission_N3CC.py`. Now, we will show how to create an N3CC example in level3. The key is that we follow the same steps:
- init
- load_inputs
- check_and_preprocess_inputs (optional)
- add_pre_mission_systems
- add_phases
- add_post_mission_systems
- link_phases
- add_driver
- add_design_variables (optional)
- add_objective
- setup
- set_initial_guesses
- run_aviary_problem

### Examining an N3CC design case

In [7]:
'''
NOTES:
Includes:
Takeoff, Climb, Cruise, Descent, Landing
Computed Aero
N3CC data
'''

import dymos as dm
import openmdao.api as om
import scipy.constants as _units

import aviary.api as av


prob = om.Problem(model=om.Group())
driver = prob.driver = om.pyOptSparseDriver()
driver.options["optimizer"] = "IPOPT"
driver.opt_settings["max_iter"] = 100
driver.opt_settings["tol"] = 1e-3
driver.opt_settings['print_level'] = 4

##########################################
# Aircraft Input Variables and Options   #
##########################################

aviary_inputs = av.get_flops_inputs('N3CC')

aviary_inputs.set_val(av.Mission.Landing.LIFT_COEFFICIENT_MAX,
                        2.4, units="unitless")
aviary_inputs.set_val(av.Mission.Takeoff.LIFT_COEFFICIENT_MAX,
                        2.0, units="unitless")
aviary_inputs.set_val(
    av.Mission.Takeoff.ROLLING_FRICTION_COEFFICIENT,
    val=.0175, units="unitless")

takeoff_fuel_burned = 577  # lbm TODO: where should this get connected from?
takeoff_thrust_per_eng = 24555.5  # lbf TODO: where should this get connected from?
takeoff_L_over_D = 17.35  # TODO: should this come from aero?

aviary_inputs.set_val(av.Mission.Takeoff.FUEL_SIMPLE,
                        takeoff_fuel_burned, units='lbm')
aviary_inputs.set_val(av.Mission.Takeoff.LIFT_OVER_DRAG,
                        takeoff_L_over_D, units="unitless")
aviary_inputs.set_val(av.Mission.Design.THRUST_TAKEOFF_PER_ENG,
                        takeoff_thrust_per_eng, units='lbf')

alt_airport = 0  # ft
cruise_mach = 0.79

alt_i_climb = 0*_units.foot  # m
alt_f_climb = 35000.0*_units.foot  # m
mass_i_climb = 131000*_units.lb  # kg
mass_f_climb = 126000*_units.lb  # kg
# initial mach set to lower value so it can intersect with takeoff end mach
# mach_i_climb = 0.3
mach_i_climb = 0.2
mach_f_climb = cruise_mach
range_i_climb = 0*_units.nautical_mile  # m
range_f_climb = 160.3*_units.nautical_mile  # m
t_i_climb = 2 * _units.minute  # sec
t_f_climb = 26.20*_units.minute  # sec
t_duration_climb = t_f_climb - t_i_climb

alt_i_cruise = 35000*_units.foot  # m
alt_f_cruise = 35000*_units.foot  # m
alt_min_cruise = 35000*_units.foot  # m
alt_max_cruise = 35000*_units.foot  # m
mass_i_cruise = 126000*_units.lb  # kg
mass_f_cruise = 102000*_units.lb  # kg
cruise_mach = cruise_mach
range_i_cruise = 160.3*_units.nautical_mile  # m
range_f_cruise = 3243.9*_units.nautical_mile  # m
t_i_cruise = 26.20*_units.minute  # sec
t_f_cruise = 432.38*_units.minute  # sec
t_duration_cruise = t_f_cruise - t_i_cruise

alt_i_descent = 35000*_units.foot
# final altitude set to 35 to ensure landing is feasible point
# alt_f_descent = 0*_units.foot
alt_f_descent = 35*_units.foot
mach_i_descent = 0.79
mach_f_descent = 0.3
mass_i_descent = 102000*_units.pound
mass_f_descent = 101000*_units.pound
distance_i_descent = 3243.9*_units.nautical_mile
distance_f_descent = 3378.7*_units.nautical_mile
t_i_descent = 432.38*_units.minute
t_f_descent = 461.62*_units.minute
t_duration_descent = t_f_descent - t_i_descent

##########################
# Design Variables       #
##########################

# Nudge it a bit off the correct answer to verify that the optimize takes us there.
aviary_inputs.set_val(av.Mission.Design.GROSS_MASS, 135000.0, units='lbm')

prob.model.add_design_var(av.Mission.Design.GROSS_MASS, units='lbm',
                            lower=100000.0, upper=200000.0, ref=135000)

takeoff_options = av.HeightEnergyTakeoffPhaseBuilder(
    airport_altitude=alt_airport,  # ft
    # no units
    num_engines=aviary_inputs.get_val(av.Aircraft.Engine.NUM_ENGINES)
)

##################
# Define Phases  #
##################
num_segments_climb = 6
num_segments_cruise = 1
num_segments_descent = 5

climb_seg_ends, _ = dm.utils.lgl.lgl(num_segments_climb + 1)
descent_seg_ends, _ = dm.utils.lgl.lgl(num_segments_descent + 1)

transcription_climb = dm.Radau(
    num_segments=num_segments_climb, order=3, compressed=True,
    segment_ends=climb_seg_ends)
transcription_cruise = dm.Radau(
    num_segments=num_segments_cruise, order=3, compressed=True)
transcription_descent = dm.Radau(
    num_segments=num_segments_descent, order=3, compressed=True,
    segment_ends=descent_seg_ends)

climb_options = av.HeightEnergyPhaseBuilder(
    'test_climb',
    user_options=av.AviaryValues({
        'initial_altitude': (alt_i_climb, 'm'),
        'final_altitude': (alt_f_climb, 'm'),
        'initial_mach': (mach_i_climb, 'unitless'),
        'final_mach': (mach_f_climb, 'unitless'),
        'fix_initial': (False, 'unitless'),
        'input_initial': (True, 'unitless'),
        'use_polynomial_control': (False, 'unitless'),
    }),
    core_subsystems=av.default_mission_subsystems,
    subsystem_options={'core_aerodynamics': {'method': 'computed'}},
    transcription=transcription_climb,
)

cruise_options = av.HeightEnergyPhaseBuilder(
    'test_cruise',
    user_options=av.AviaryValues({
        'initial_altitude': (alt_min_cruise, 'm'),
        'final_altitude': (alt_max_cruise, 'm'),
        'initial_mach': (cruise_mach, 'unitless'),
        'final_mach': (cruise_mach, 'unitless'),
        'required_available_climb_rate': (300, 'ft/min'),
        'fix_initial': (False, 'unitless'),
    }),
    core_subsystems=av.default_mission_subsystems,
    subsystem_options={'core_aerodynamics': {'method': 'computed'}},
    transcription=transcription_cruise,
)

descent_options = av.HeightEnergyPhaseBuilder(
    'test_descent',
    user_options=av.AviaryValues({
        'final_altitude': (alt_f_descent, 'm'),
        'initial_altitude': (alt_i_descent, 'm'),
        'initial_mach': (mach_i_descent, 'unitless'),
        'final_mach': (mach_f_descent, 'unitless'),
        'fix_initial': (False, 'unitless'),
        'use_polynomial_control': (False, 'unitless'),
    }),
    core_subsystems=av.default_mission_subsystems,
    subsystem_options={'core_aerodynamics': {'method': 'computed'}},
    transcription=transcription_descent,
)

landing_options = av.HeightEnergyLandingPhaseBuilder(
    ref_wing_area=aviary_inputs.get_val(av.Aircraft.Wing.AREA, units='ft**2'),
    Cl_max_ldg=aviary_inputs.get_val(
        av.Mission.Landing.LIFT_COEFFICIENT_MAX)  # no units
)

av.preprocess_crewpayload(aviary_inputs)

# Upstream static analysis for aero
prob.model.add_subsystem(
    'pre_mission',
    av.CorePreMission(aviary_options=aviary_inputs,
                    subsystems=av.default_premission_subsystems),
    promotes_inputs=['aircraft:*', 'mission:*'],
    promotes_outputs=['aircraft:*', 'mission:*'])

# directly connect phases (strong_couple = True), or use linkage constraints (weak
# coupling / strong_couple=False)
strong_couple = False

takeoff = takeoff_options.build_phase(False)

climb = climb_options.build_phase(aviary_options=aviary_inputs)

cruise = cruise_options.build_phase(aviary_options=aviary_inputs)

descent = descent_options.build_phase(aviary_options=aviary_inputs)

landing = landing_options.build_phase(False)

prob.model.add_subsystem(
    'takeoff', takeoff, promotes_inputs=['aircraft:*', 'mission:*'],
    promotes_outputs=['mission:*'])

traj = prob.model.add_subsystem('traj', dm.Trajectory())

# if fix_initial is false, can we always set input_initial to be true for
# necessary states, and then ignore if we use a linkage?
climb.set_time_options(fix_initial=True, fix_duration=False, units='s',
                        duration_bounds=(t_duration_climb*0.5, t_duration_climb*2),
                        duration_ref=t_duration_climb)
cruise.set_time_options(fix_initial=False, fix_duration=False, units='s',
                        duration_bounds=(t_duration_cruise*0.5, t_duration_cruise*2),
                        duration_ref=t_duration_cruise,
                        initial_bounds=(t_duration_climb*0.5, t_duration_climb*2))
descent.set_time_options(
    fix_initial=False, fix_duration=False, units='s',
    duration_bounds=(t_duration_descent*0.5, t_duration_descent*2),
    duration_ref=t_duration_descent,
    initial_bounds=(
        (t_duration_cruise + t_duration_climb)*0.5,
        (t_duration_cruise + t_duration_climb)*2))

traj.add_phase('climb', climb)

traj.add_phase('cruise', cruise)

traj.add_phase('descent', descent)

prob.model.add_subsystem(
    'landing', landing, promotes_inputs=['aircraft:*', 'mission:*'],
    promotes_outputs=['mission:*'])

##########################################
# link phases                            #
##########################################

traj.link_phases(["climb", "cruise", "descent"], ["time", av.Dynamic.Mission.MASS, av.Dynamic.Mission.DISTANCE], connected=strong_couple)

traj = av.setup_trajectory_params(prob.model, traj, aviary_inputs)

##################################
# Connect in Takeoff and Landing #
##################################
prob.model.connect(av.Mission.Takeoff.FINAL_MASS,
                    'traj.climb.initial_states:mass')
prob.model.connect(av.Mission.Takeoff.GROUND_DISTANCE,
                    'traj.climb.initial_states:distance')

prob.model.connect('traj.descent.states:mass',
                    av.Mission.Landing.TOUCHDOWN_MASS, src_indices=[-1])
# TODO: approach velocity should likely be connected
prob.model.connect('traj.descent.control_values:altitude', av.Mission.Landing.INITIAL_ALTITUDE,
                    src_indices=[-1])

##########################
# Constraints            #
##########################

ecomp = om.ExecComp('fuel_burned = initial_mass - descent_mass_final',
                    initial_mass={'units': 'lbm', 'shape': 1},
                    descent_mass_final={'units': 'lbm', 'shape': 1},
                    fuel_burned={'units': 'lbm', 'shape': 1})

prob.model.add_subsystem('fuel_burn', ecomp,
                            promotes_inputs=[
                                ('initial_mass', av.Mission.Design.GROSS_MASS)],
                            promotes_outputs=['fuel_burned'])

prob.model.connect("traj.descent.states:mass",
                    "fuel_burn.descent_mass_final", src_indices=[-1])

# TODO: need to add some sort of check that this value is less than the fuel capacity
# TODO: need to update this with actual FLOPS value, this gives unrealistic
# appearance of accuracy
# TODO: the overall_fuel variable is the burned fuel plus the reserve, but should
# also include the unused fuel, and the hierarchy variable name should be more clear
ecomp = om.ExecComp('overall_fuel = fuel_burned + fuel_reserve',
                    fuel_burned={'units': 'lbm', 'shape': 1},
                    fuel_reserve={'units': 'lbm', 'val': 2173.},
                    overall_fuel={'units': 'lbm'})
prob.model.add_subsystem('fuel_calc', ecomp,
                            promotes_inputs=['fuel_burned'],
                            promotes_outputs=['overall_fuel'])

ecomp = om.ExecComp(
    'mass_resid = operating_empty_mass + overall_fuel + payload_mass '
    '- initial_mass',
    operating_empty_mass={'units': 'lbm'},
    overall_fuel={'units': 'lbm'},
    payload_mass={'units': 'lbm'},
    initial_mass={'units': 'lbm'},
    mass_resid={'units': 'lbm'})

prob.model.add_subsystem(
    'mass_constraint', ecomp,
    promotes_inputs=[
        ('operating_empty_mass', av.Aircraft.Design.OPERATING_MASS),
        'overall_fuel',
        ('payload_mass', av.Aircraft.CrewPayload.TOTAL_PAYLOAD_MASS),
        ('initial_mass', av.Mission.Design.GROSS_MASS)],
    promotes_outputs=['mass_resid'])

prob.model.add_constraint('mass_resid', equals=0.0, ref=1.0)

##########################
# Add Objective Function #
##########################

# This is an example of a overall mission objective
# create a compound objective that minimizes climb time and maximizes final mass
# we are maxing final mass b/c we don't have an independent value for fuel_mass yet
# we are going to normalize these (making each of the sub-objectives approx = 1 )
# TODO: change the scaling on climb_duration
prob.model.add_subsystem(
    "regularization",
    om.ExecComp(
        "reg_objective = fuel_mass/1500",
        reg_objective=0.0,
        fuel_mass={"units": "lbm", "shape": 1},
    ),
    promotes_outputs=['reg_objective']
)
# connect the final mass from cruise into the objective
prob.model.connect(av.Mission.Design.FUEL_MASS, "regularization.fuel_mass")

prob.model.add_objective('reg_objective', ref=1)

# Set initial default values for all LEAPS aircraft variables.
av.set_aviary_initial_values(prob.model, aviary_inputs)

prob.model.add_subsystem(
    'input_sink',
    av.VariablesIn(aviary_options=aviary_inputs),
    promotes_inputs=['*'],
    promotes_outputs=['*']
)

prob.setup(force_alloc_complex=True)

###########################################
# Intial Settings for States and Controls #
###########################################

prob.set_val('traj.climb.t_initial', t_i_climb, units='s')
prob.set_val('traj.climb.t_duration', t_duration_climb, units='s')

prob.set_val('traj.climb.controls:altitude', climb.interp(
    av.Dynamic.Mission.ALTITUDE, ys=[alt_i_climb, alt_f_climb]), units='m')
prob.set_val(
    'traj.climb.controls:mach', climb.interp(
        av.Dynamic.Mission.MACH, ys=[mach_i_climb, mach_f_climb]), units='unitless')
prob.set_val('traj.climb.states:mass', climb.interp(
    av.Dynamic.Mission.MASS, ys=[mass_i_climb, mass_f_climb]), units='kg')
prob.set_val('traj.climb.states:distance', climb.interp(
    av.Dynamic.Mission.DISTANCE, ys=[range_i_climb, range_f_climb]), units='m')

prob.set_val('traj.cruise.t_initial', t_i_cruise, units='s')
prob.set_val('traj.cruise.t_duration', t_duration_cruise, units='s')

prob.set_val('traj.cruise.polynomial_controls:altitude', cruise.interp(
    av.Dynamic.Mission.ALTITUDE, ys=[alt_i_cruise, alt_f_cruise]), units='m')
prob.set_val(
    'traj.cruise.polynomial_controls:mach', cruise.interp(
        av.Dynamic.Mission.MACH, ys=[cruise_mach, cruise_mach]), units='unitless')
prob.set_val('traj.cruise.states:mass', cruise.interp(
    av.Dynamic.Mission.MASS, ys=[mass_i_cruise, mass_f_cruise]), units='kg')
prob.set_val('traj.cruise.states:distance', cruise.interp(
    av.Dynamic.Mission.DISTANCE, ys=[range_i_cruise, range_f_cruise]), units='m')

prob.set_val('traj.descent.t_initial', t_i_descent, units='s')
prob.set_val('traj.descent.t_duration', t_duration_descent, units='s')

prob.set_val('traj.descent.controls:altitude', descent.interp(
    av.Dynamic.Mission.ALTITUDE, ys=[alt_i_descent, alt_f_descent]), units='m')
prob.set_val(
    'traj.descent.controls:mach', descent.interp(
        av.Dynamic.Mission.MACH, ys=[mach_i_descent, mach_f_descent]), units='unitless')
prob.set_val('traj.descent.states:mass', descent.interp(
    av.Dynamic.Mission.MASS, ys=[mass_i_descent, mass_f_descent]), units='kg')
prob.set_val('traj.descent.states:distance', descent.interp(
    av.Dynamic.Mission.DISTANCE, ys=[distance_i_descent, distance_f_descent]), units='m')

# Turn off solver printing so that the SNOPT output is readable.
prob.set_solver_print(level=0)

dm.run_problem(prob, simulate=False, make_plots=False, simulate_kwargs={
                'times_per_seg': 100, 'atol': 1e-9, 'rtol': 1e-9},
                solution_record_file='N3CC_sizing.db')
prob.record("final")
prob.cleanup()

times_climb = prob.get_val('traj.climb.timeseries.time', units='s')
altitudes_climb = prob.get_val(
    'traj.climb.timeseries.altitude', units='m')
masses_climb = prob.get_val('traj.climb.timeseries.mass', units='kg')
ranges_climb = prob.get_val('traj.climb.timeseries.distance', units='m')
velocities_climb = prob.get_val(
    'traj.climb.timeseries.velocity', units='m/s')
thrusts_climb = prob.get_val('traj.climb.timeseries.thrust_net_total', units='N')
times_cruise = prob.get_val('traj.cruise.timeseries.time', units='s')
altitudes_cruise = prob.get_val(
    'traj.cruise.timeseries.altitude', units='m')
masses_cruise = prob.get_val('traj.cruise.timeseries.mass', units='kg')
ranges_cruise = prob.get_val('traj.cruise.timeseries.distance', units='m')
velocities_cruise = prob.get_val(
    'traj.cruise.timeseries.velocity', units='m/s')
thrusts_cruise = prob.get_val(
    'traj.cruise.timeseries.thrust_net_total', units='N')
times_descent = prob.get_val('traj.descent.timeseries.time', units='s')
altitudes_descent = prob.get_val(
    'traj.descent.timeseries.altitude', units='m')
masses_descent = prob.get_val('traj.descent.timeseries.mass', units='kg')
ranges_descent = prob.get_val('traj.descent.timeseries.distance', units='m')
velocities_descent = prob.get_val(
    'traj.descent.timeseries.velocity', units='m/s')
thrusts_descent = prob.get_val(
    'traj.descent.timeseries.thrust_net_total', units='N')


print("-------------------------------")
print(f"times_climb: {times_climb[-1]} (s)")
print(f"altitudes_climb: {altitudes_climb[-1]} (m)")
print(f"masses_climb: {masses_climb[-1]} (kg)")
print(f"ranges_climb: {ranges_climb[-1]} (m)")
print(f"velocities_climb: {velocities_climb[-1]} (m/s)")
print(f"thrusts_climb: {thrusts_climb[-1]} (N)")
print(f"times_cruise: {times_cruise[-1]} (s)")
print(f"altitudes_cruise: {altitudes_cruise[-1]} (m)")
print(f"masses_cruise: {masses_cruise[-1]} (kg)")
print(f"ranges_cruise: {ranges_cruise[-1]} (m)")
print(f"velocities_cruise: {velocities_cruise[-1]} (m/s)")
print(f"thrusts_cruise: {thrusts_cruise[-1]} (N)")
print(f"times_descent: {times_descent[-1]} (s)")
print(f"altitudes_descent: {altitudes_descent[-1]} (m)")
print(f"masses_descent: {masses_descent[-1]} (kg)")
print(f"ranges_descent: {ranges_descent[-1]} (m)")
print(f"velocities_descent: {velocities_descent[-1]} (m/s)")
print(f"thrusts_descent: {thrusts_descent[-1]} (N)")
print("-------------------------------")



The following variables have been overridden:
  'aircraft:design:landing_to_takeoff_mass_ratio
  'aircraft:engine:mass
  'aircraft:fuel:auxiliary_fuel_capacity
  'aircraft:fuel:fuselage_fuel_capacity
  'aircraft:fuselage:planform_area
  'aircraft:fuselage:wetted_area
  'aircraft:horizontal_tail:wetted_area
  'aircraft:nacelle:total_wetted_area
  'aircraft:nacelle:wetted_area
  'aircraft:propulsion:total_engine_controls_mass
  'aircraft:propulsion:total_starter_mass
  'aircraft:vertical_tail:wetted_area
  'aircraft:wing:aspect_ratio
  'aircraft:wing:wetted_area

--- Constraint Report [traj] ---
    --- climb ---
        None
    --- cruise ---
        [path]    3.0000e+02 <= altitude_rate_max  [ft/min]
    --- descent ---
        None



/mnt/c/Users/John/Dropbox/git/OpenMDAO/openmdao/core/system.py:2431: PromotionWarning:'pre_mission.core_geometry.canard' <class Canard>: input variable 'aircraft:canard:area', promoted using 'aircraft:canard:area', was already promoted using 'aircraft*'.
/mnt/c/Users/John/Dropbox/git/OpenMDAO/openmdao/core/system.py:2431: PromotionWarning:'pre_mission.core_geometry.canard' <class Canard>: input variable 'aircraft:canard:thickness_to_chord', promoted using 'aircraft:canard:thickness_to_chord', was already promoted using 'aircraft*'.
/mnt/c/Users/John/Dropbox/git/OpenMDAO/openmdao/core/system.py:2431: PromotionWarning:'pre_mission.core_geometry.canard' <class Canard>: input variable 'aircraft:canard:wetted_area_scaler', promoted using 'aircraft:canard:wetted_area_scaler', was already promoted using 'aircraft*'.
/mnt/c/Users/John/Dropbox/git/OpenMDAO/openmdao/core/system.py:2431: PromotionWarning:'pre_mission.core_geometry.characteristic_lengths' <class CharacteristicLengths>: input varia

Model viewer data has already been recorded for Driver.

List of user-set options:

                                    Name   Value                used
                        file_print_level = 5                     yes
                   hessian_approximation = limited-memory        yes
                           linear_solver = mumps                 yes
                                max_iter = 100                   yes
                      nlp_scaling_method = user-scaling          yes
                             output_file = reports/problem7/IPOPT.out  yes
                             print_level = 4                     yes
                      print_user_options = yes                   yes
                                      sb = yes                   yes
                                     tol = 0.001                 yes
Total number of variables............................:       82
                     variables with only lower bounds:        0
                variabl

This model demonstrates the flexibility of level 3. For example, we do not load the aircraft model from a `.csv` file but from a Python file using the `get_flops_inputs()` method.

This function not only reads Aviary and mission variables but also builds the engine. More information can be found in `aviary/models/N3CC/N3CC_data.py`.

Note that we can read large single aisle aircraft inputs this way as well:

In [8]:
aviary_inputs = av.get_flops_inputs('LargeSingleAisle1FLOPS')
aviary_inputs = av.get_flops_inputs('LargeSingleAisle2FLOPS')
aviary_inputs = av.get_flops_inputs('LargeSingleAisle2FLOPSdw')
aviary_inputs = av.get_flops_inputs('LargeSingleAisle2FLOPSalt')

The data files are at:
```
aviary/models/large_single_aisle_1/large_single_aisle_1_FLOPS_data.py
aviary/models/large_single_aisle_2/large_single_aisle_2_FLOPS_data.py
aviary/models/large_single_aisle_2/large_single_aisle_2_detailwing_FLOPS_data.py
aviary/models/large_single_aisle_2/large_single_aisle_2_altwt_FLOPS_data.py
```

respectively.

### Discussing this example in more detail

We move all the code blocks on taxi to `add_pre_mission_system` function because it is how it is done in `methods_for_level2.py`. Similarly, all the code blocks on landing are moved to `add_post_mission_systems` function. Be careful! Generally speaking, not all components can be moved around due to the expected order of execution.

In `aviary/validation_cases/benchmark_tests` folder, there is another N3CC model `test_FLOPS_based_sizing_N3CC.py`. If we had started from that model, you would need to have an `add_design_variables` function. The last function `print_outputs` is added in our model to verify that we didn't alter the original model. It is not a level 2 function but it is a good idea to have this function anyway.

## Summary

We have shown in an example that users have total control over every aspect of the model in level 3. There is one big feature that hasn't been covered: adding external subsystems. We will move on to discuss how to run [Aviary External Subsystem](onboarding_ext_subsystem) next.